In [2]:
import requests
from timezonefinder import TimezoneFinder
import pytz
import datetime
import os
import requests
from PIL import Image, ImageFont, ImageDraw
from IPython.display import display
from io import BytesIO


In [86]:
TAMPERE = 61.497753, 23.760954
STOCKHOLM = 59.334591, 18.063240

In [4]:
def get_timezone(coords):
    tf = TimezoneFinder()
    latitude, longitude = coords
    timezone_name = tf.timezone_at(lng=longitude, lat=latitude)
    timezone_object = pytz.timezone(timezone_name)
    return timezone_object

In [54]:
def get_daytime_data(coords, date='today'):
    lat, long = coords
    sunrise_sunset_api_url = f'https://api.sunrise-sunset.org/json?lat={lat}&lng={long}&formatted=0&date={date}'
    return requests.get(sunrise_sunset_api_url).json()


In [8]:
def parse_to_local_time(iso8601, tz):
    dt = datetime.datetime.fromisoformat(iso8601)
    return pytz.utc.normalize(dt).astimezone(tz)

In [84]:
def day_length_info_string(coords):
    timezone = get_timezone(coords)
    today_daytime_data = get_daytime_data(coords)
    sunrise_today = parse_to_local_time(today_daytime_data['results']['sunrise'], timezone)
    sunset_today = parse_to_local_time(today_daytime_data['results']['sunset'], timezone)

    day_length_today = datetime.timedelta(seconds=today_daytime_data['results']['day_length'])

    yesterday_daytime_data = get_daytime_data(coords, date=datetime.datetime.today() - datetime.timedelta(1))
    day_length_yesterday = datetime.timedelta(seconds=yesterday_daytime_data['results']['day_length'])

    day_before_yesterday_daytime_data = get_daytime_data(coords, date=datetime.datetime.today() - datetime.timedelta(2))
    day_length_before_yesterday = datetime.timedelta(seconds=day_before_yesterday_daytime_data['results']['day_length'])

    sunrise_today_str = sunrise_today.strftime('%H:%M:%S')
    sunset_today_str = sunset_today.strftime('%H:%M:%S')
    change_str = str(day_length_today - day_length_yesterday)
    delta = abs((day_length_today - day_length_yesterday) - (day_length_yesterday - day_length_before_yesterday))
    
    return (f"Nousu: {sunrise_today_str:>10}\n"
            f"Lasku: {sunset_today_str:>10}\n"
            f"Pituus: {str(day_length_today):>9}\n"
            f"Muutos: {change_str:>9}\n"
            f"Delta: {str(delta):>10}")
    


In [150]:
img = Image.new('RGB', (750, 350), color='white')
draw = ImageDraw.Draw(img)
font_heading = ImageFont.truetype("Helvetica.ttc", 48)
font_body = ImageFont.truetype("jetbrains-mono.ttf", 32)

draw.text((75, 10),"Tampere",(0,0,0), font=font_heading)
draw.text((475, 10), "Stockholm",(0,0,0), font=font_heading)

draw.text((0, 80), day_length_info_string(TAMPERE) ,(0,0,0), font=font_body)
draw.text((400, 80), day_length_info_string(STOCKHOLM) ,(0,0,0), font=font_body)

flag_finland = Image.open('flag-finland.png', 'r').resize((64, 64), Image.ANTIALIAS)
img.paste(flag_finland, (0,0), flag_finland)

flag_sweden = Image.open('flag-sweden.png', 'r').resize((64, 64), Image.ANTIALIAS)
img.paste(flag_sweden, (400,0), flag_sweden)

fd = BytesIO()
img.save(fd, "png", quality=95)


In [158]:
os.environ['BOT_KEY'] = 'bot700928179:AAGJuu8ubwHhpYhkG5LzlFJzglafsyQoJmw'
os.environ['CHAT_ID'] = '-248024980'

In [153]:
fd.seek(0)
files = {'photo': fd.read()}

In [159]:
data = {'chat_id': os.environ['CHAT_ID']}

In [155]:
url = f"https://api.telegram.org/{os.environ['BOT_KEY']}/sendPhoto"

In [156]:
requests.post(url, data=data, files=files).content

b'{"ok":true,"result":{"message_id":1376,"from":{"id":700928179,"is_bot":true,"first_name":"Yleisbotti","username":"vuokrabot"},"chat":{"id":-248024980,"title":"AURINKOA CHAT \\u2600","type":"group","all_members_are_administrators":true},"date":1581169277,"photo":[{"file_id":"AgACAgQAAxkDAAIFYF4-un0i86OYdCLd2IKpGyhbLOJtAAIDsTEbmpH5Ue91vmkHNWLPoEi2GwAEAQADAgADbQADYNUCAAEYBA","file_unique_id":"AQADoEi2GwAEYNUCAAE","file_size":15755,"width":320,"height":149},{"file_id":"AgACAgQAAxkDAAIFYF4-un0i86OYdCLd2IKpGyhbLOJtAAIDsTEbmpH5Ue91vmkHNWLPoEi2GwAEAQADAgADeAADYdUCAAEYBA","file_unique_id":"AQADoEi2GwAEYdUCAAE","file_size":46194,"width":750,"height":350}]}}'